In [1]:
import org.apache.spark.sql.{SparkSession, DataFrame}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

// Initialize SparkSession
val spark = SparkSession.builder()
  .appName("Duplicate Record Removal Pipeline")
  .getOrCreate()

spark = org.apache.spark.sql.SparkSession@2d61ac14


org.apache.spark.sql.SparkSession@2d61ac14

In [19]:
val moviesPath = "gs://bhargav-assignments/Day16And17Task/movie.csv"  // GCP Cloud Storage path
val hdfsOutputPath = "hdfs:///user/Day16AndDay17/case_study_4/movies_from_gcp.csv"  // HDFS output path
//GCP Cloud Storage path for Avro output
val avroOutputPath = "gs://bhargav-assignments/Day16And17Task/case_study_4/cleaned_movies_avro"

// Load movies.csv from GCP Cloud Storage into a DataFrame
val moviesDF = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv(moviesPath)

// Show the first few rows for validation
moviesDF.show(5, truncate = false)


+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows



moviesPath = gs://bhargav-assignments/Day16And17Task/movie.csv
hdfsOutputPath = hdfs:///user/Day16AndDay17/case_study_4/movies_from_gcp.csv
avroOutputPath = gs://bhargav-assignments/Day16And17Task/case_study_4/cleaned_movies_avro
moviesDF = [movieId: int, title: string ... 1 more field]


[movieId: int, title: string ... 1 more field]

In [16]:
// Create duplicate records
val duplicateMoviesDF = spark.createDataFrame(Seq(
  ("1", "Toy Story (1995)", "Adventure|Animation|Children|Comedy|Fantasy"),
  ("1", "Toy Story (1995)", "Adventure|Animation|Children|Comedy|Fantasy"),
  ("2", "Jumanji (1995)", "Adventure|Children|Fantasy"),
  ("2", "Jumanji (1995)", "Adventure|Children|Fantasy")
)).toDF("movieId", "title", "genres")

// Append the duplicate records to the original DataFrame
val updatedMoviesDF = moviesDF.union(duplicateMoviesDF)

// Write the DataFrame to HDFS
updatedMoviesDF.write
  .mode("overwrite")
  .option("header", "true")
  .csv(hdfsOutputPath)

// Successfully saved to HDFS
println(s"Movies data saved to HDFS path: $hdfsOutputPath")

Movies data saved to HDFS path: hdfs:///user/Day16AndDay17/case_study_4/movies_from_gcp.csv


duplicateMoviesDF = [movieId: string, title: string ... 1 more field]
updatedMoviesDF = [movieId: string, title: string ... 1 more field]


[movieId: string, title: string ... 1 more field]

In [17]:
// Load the movies.csv file from HDFS
val moviesFromHDFSDf = spark.read
  .option("header", "true")
  .option("inferSchema", "true")
  .csv(hdfsOutputPath)

moviesFromHDFSDf.show(5, truncate = false)

// Show initial record count before removing duplicates
val initialCount = moviesFromHDFSDf.count()
println(s"Initial count of records: $initialCount")

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows

Initial count of records: 27282


moviesFromHDFSDf = [movieId: int, title: string ... 1 more field]
initialCount = 27282


27282

In [22]:
// Remove duplicates based on movieId and title using DataFrame API
val cleanedMoviesDF = moviesFromHDFSDf
  .dropDuplicates("movieId", "title")

// Show the record count after removing duplicates
val cleanedCount = cleanedMoviesDF.count()
println(s"Count after removing duplicates: $cleanedCount")

// Validate that duplicates were removed
val duplicatesRemoved = initialCount - cleanedCount
println(s"Number of duplicates removed: $duplicatesRemoved")

Count after removing duplicates: 27278
Number of duplicates removed: 4


lastException = null
cleanedMoviesDF = [movieId: int, title: string ... 1 more field]
cleanedCount = 27278
duplicatesRemoved = 4


4

In [25]:
// Convert the cleaned DataFrame to an RDD
val cleanedMoviesRDD = cleanedMoviesDF.rdd

// Apply distinct on composite key (movieId, title)
val distinctMoviesRDD = cleanedMoviesRDD.map(row => (row.getAs[Int]("movieId"), row.getAs[String]("title")))
  .distinct()

// Convert back to DataFrame
val finalCleanedMoviesDF = distinctMoviesRDD.toDF("movieId", "title")

// Show the final cleaned DataFrame
finalCleanedMoviesDF.show(5, truncate = false)

+-------+-----------------------------------------------------------+
|movieId|title                                                      |
+-------+-----------------------------------------------------------+
|82108  |Against the Current (2009)                                 |
|94133  |Hammer, The (2010)                                         |
|91284  |Lonely Passion of Judith Hearne, The (1987)                |
|96717  |Pearls of the Crown, The (Les perles de la couronne) (1937)|
|69042  |Flash Gordon's Trip to Mars (1938)                         |
+-------+-----------------------------------------------------------+
only showing top 5 rows



cleanedMoviesRDD = MapPartitionsRDD[295] at rdd at <console>:25
distinctMoviesRDD = MapPartitionsRDD[313] at distinct at <console>:31
finalCleanedMoviesDF = [movieId: int, title: string]


[movieId: int, title: string]

In [26]:
// Save the cleaned DataFrame in Avro format to GCP Cloud Storage
finalCleanedMoviesDF.write
  .mode("overwrite")
  .format("avro")
  .save(avroOutputPath)

println(s"Cleaned movies data saved to Avro format at $avroOutputPath")

Cleaned movies data saved to Avro format at gs://bhargav-assignments/Day16And17Task/case_study_4/cleaned_movies_avro


In [27]:
// Stop the Spark session
spark.stop()